In [1]:
# Applying Bender's decomposition for AoI_energy UAV path planning problem

# Reference: https://github.com/samarthmistry/Large-scale-Supply-Chain-Optimization
# The file 'inputdata.py' is from the Cplex example folder 
# to input the '.dat' data: https://www.ibm.com/uk-en/analytics/cplex-optimizer

# Gurobi version: gurobi 9.1.2

# The coefficient metrices have been calculated before and saved as '.dat' files

from inputdata import read_dat_file
import gurobipy as gp
from gurobipy import GRB
import numpy as np

# upload the coefficients
T_coef = read_dat_file("T_coef.dat")[0]
E_coef= read_dat_file("E_coef.dat")[0]
num_BS_SN = len(T_coef)
num_SN = num_BS_SN - 1

def bendersCallback(model, where):
    
    # Found a new (integer) solution to master problem, so use callback
    if where == GRB.callback.MIPSOL:
        
        # Extract 'x' values from master problem solution
        x_bar = []
        for i in range(num_BS_SN):
            x_bar.append([])
            for j in range(num_BS_SN):
                x_bar_value = model.cbGetSolution(model.getVarByName('x['+str(i)+','+str(j)+']'))
                x_bar[i].append(x_bar_value)

        # Initialize dual subproblem instance
        sub = gp.Model('DualSubproblem')
        
        # dual subproblem
        # maximize sum(i in K) u(i) - sum((i,j) in E) num_SN * x(i,j) * w(i,j)
        # Constraints:
        # forall i in K: -u(i) + v(i) - w(0,i) <= T_coef(0,i)
        # forall i in K:  u(i)        - w(i,0) <= T_coef(i,0)
        # forall (i,j) in E, i!=0, j!= 0: u(i) - u(j) - w(i,j) <= T_coef(i,j)
        # u,v free, w >= 0
        
        # Add dual variables
        u = sub.addVars(num_SN, obj = 0.0, vtype=GRB.CONTINUOUS, name = "u")
        v = sub.addVars(num_SN, obj = 0.0, vtype=GRB.CONTINUOUS, name = "v")
        w = sub.addVars(num_BS_SN, num_BS_SN, lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name = "w")
        
        for i in range(num_BS_SN):
            w[i,i].setAttr("ub", 0.0)
            
        sub.update()
        
        # Add dual constraints
        for i in range(1,num_BS_SN):
            uv_index = i - 1
            sub.addConstr( -1.0 * u[uv_index] + v[uv_index] - w[0,i] <= T_coef[0][i], "DualCons1[%d]" % i)
            sub.addConstr(u[uv_index] - w[i,0] <= T_coef[i][0], "DualCons2[%d]" % i)
        for i in range(1,num_BS_SN):
            i_index = i - 1
            for j in range(1,num_BS_SN):
                j_index = j - 1
                sub.addConstr(u[i_index] - u[j_index] - w[i,j] <= T_coef[i][j], "")
                
        # Initialize objective function
        obj_sub = gp.LinExpr()
        for i in range(num_SN):
            obj_sub += u[i]
        for i in range(num_BS_SN):
            for j in range(num_BS_SN):
                obj_sub += -1.0 * num_SN * x_bar[i][j] * w[i,j]
        
        # Set objective
        sub.setObjective(obj_sub, GRB.MAXIMIZE)
        
        # Gurobi params
        sub.setParam('LogToConsole', 0)
        sub.setParam('InfUnbdInfo', 1)
        sub.setParam('DualReductions', 0)
        
        # Solve dual subproblem
        sub.optimize()
        
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #     BENDERS' FEASIBILITY CUT
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # If subproblem is unbounded
        # Compute the cut from the unbounded ray. The cut is:
        # sum(i in K) u(i) - sum((i,j) in E) num_SN * x(i,j) * w(i,j) <= 0
        if sub.status == 5:

            # Extract Unbounded Rays 'u' & 'w' values from solution of subproblem
            u_bar = np.zeros(num_SN)
            for i in range(num_SN):
                u_bar[i] = u[i].UnbdRay

            w_bar = np.zeros((num_BS_SN, num_BS_SN))
            for i in range(num_BS_SN):
                for j in range(num_BS_SN):
                    w_bar[i,j] = w[i,j].UnbdRay

            # Set-up cut
            expr = gp.LinExpr()
            for i in range(num_SN):
                expr += u_bar[i]
            for i in range(num_BS_SN):
                for j in range(num_BS_SN):
                    expr += -1.0 * num_SN * w_bar[i,j] * x[i,j]
            
            # Add Benders' feasibility cut to master problem
            master.cbLazy(expr <= 0)

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        #     BENDERS' OPTIMALITY CUT
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # If not unbounded, then subproblem is feasible
        # Compute the optimlity cut from the extreme. The cut is:
        # sum(i in K) u(i) - sum((i,j) in E) num_SN * x(i,j) * w(i,j) <= z
        else:

            # Extract Extreme Points 'u' & 'v' values from solution of subproblem
            u_bar = np.zeros(num_SN)
            for i in range(num_SN):
                u_bar[i] = u[i].X

            w_bar = np.zeros((num_BS_SN, num_BS_SN))
            for i in range(num_BS_SN):
                for j in range(num_BS_SN):
                    w_bar[i,j] = w[i,j].X

            # Set-up cut
            expr = gp.LinExpr()
            for i in range(num_SN):
                expr += u_bar[i]
            for i in range(num_BS_SN):
                for j in range(num_BS_SN):
                    expr += -1.0 * num_SN * w_bar[i,j] * x[i,j]

            # Add Benders' optimality cut to master problem
            master.cbLazy(z >= expr)
            
# Initialize master problem instance
master = gp.Model('master_problem')

# Add binary variables
z = master.addVar(obj=1.0, name='z')
x = master.addVars(num_BS_SN, num_BS_SN, obj = E_coef, vtype=GRB.BINARY, name = "x")
master.update()

# Set objective
master.ModelSense=GRB.MINIMIZE

# Add degree constraints for depot
master.addConstr(x.sum('*', 0) ==  x.sum(0, '*') , "DepotDegree")

for i in range(1,num_BS_SN):
    # Add degree constraints for SNs
    master.addConstr(x.sum('*', i) == 1, "SNDegreeIn[%d]" % i)
    master.addConstr(x.sum(i, '*') == 1, "SNDegreeOut[%d]" % i)

# Gurobi params
master.setParam('LazyConstraints', 1)

# Solve master problem with callback solving the subproblem
master.optimize(bendersCallback)

Academic license - for non-commercial use only - expires 2021-07-26
Using license file C:\Program Files\gurobi\gurobi.lic
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 17 rows, 82 columns and 160 nonzeros
Model fingerprint: 0x29bc76d8
Variable types: 1 continuous, 81 integer (81 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 17 rows, 82 columns, 160 nonzeros
Variable types: 1 continuous, 81 integer (81 binary)

Root relaxation: objective 5.911467e+02, 17 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  658